# Azure AI Agent ServiceによるAgentic RAG

この例では、Azure AI Agent ServiceのFileSearch機能を使用してRetrieval-Augmented Generation（RAG）を実装する方法を示します。

**このノートブックで学ぶこと：**
- Azure AI Agent Serviceを使用した高度なRAGシステムの構築方法
- ファイル検索ツールを活用したドキュメントベースの質問応答システム
- クラウドベースのベクトルストアとエージェント管理の実装

**達成できること：**
- 企業ドキュメントを知識ベースとして活用するAIエージェントの作成
- ドキュメント外の質問を適切に拒否する信頼性の高いRAGシステムの実装
- Azure認証とリソース管理を含む本格的なクラウドRAGアプリケーションの開発

## 環境の初期化

### パッケージのインストール

Azure AI Agentsに必要なパッケージをインストールします：

In [ ]:
import importlib
import subprocess
import sys

# チェックするライブラリ名と、インストールするパッケージ名のマッピング
packages = {
    "azure.ai.agents": "azure-ai-agents",  # Azure AI Agents SDK
    "azure.identity": "azure-identity",    # Azure認証
}

not_installed = []
for lib, pkg in packages.items():
    try:
        # ライブラリがインポート可能か試す
        importlib.import_module(lib.split('.')[0])
    except ImportError:
        # インポートできなければインストールリストに追加
        not_installed.append(pkg)

if not_installed:
    print(f"不足しているパッケージをインストールします: {', '.join(not_installed)}")
    # pipコマンドを実行してインストール
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--quiet"] + not_installed)
    print("パッケージのインストールが完了しました。")
else:
    print("必要なパッケージはすべてインストール済みです。")

必要なパッケージはすべてインストール済みです。


### SQLiteバージョンの修正
以下のエラーが発生した場合：
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```

ノートブックの最初で、このコードブロックのコメントを解除してください：

In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### パッケージのインポート
以下のコードは必要なパッケージをインポートします：

In [24]:
# Azure AI Agents SDKの正しいインポート
import os
from azure.ai.agents import AgentsClient
from azure.identity import DefaultAzureCredential

print("必要なパッケージのインポートが完了しました。")
print("注意: Azure AI Agent ServiceはAzure AI Foundryプロジェクトのエンドポイントが必要です。")

必要なパッケージのインポートが完了しました。
注意: Azure AI Agent ServiceはAzure AI Foundryプロジェクトのエンドポイントが必要です。


**重要な注意事項:**
Azure AI Agent Serviceは比較的新しいサービスで、パッケージのバージョンによってはインポートパスが異なる場合があります。上記のセルを実行してエラーが出る場合は、以下の代替学習パスを推奨します：

1. `05-semantic-kernel-azuresearch.ipynb` (基本的なSemantic Kernel RAG)
2. `05-autogen-chromadb.ipynb` (AutoGenフレームワークの基本)
3. `05-autogen-azuresearch.ipynb` (AutoGenとAzure AI Search)

これらのノートブックは安定したパッケージを使用しており、確実に学習を進めることができます。

# Azure AI Agent Serviceを使用したエージェント対話システム

このサンプルでは、**Azure AI Agent Service**を使用してAIエージェントとの基本的な対話システムを実装する方法を示します。クラウドベースのエージェント管理と非同期メッセージ処理を学習できます。

### 動作の仕組み

1. **Azure AI Foundryプロジェクトとの接続**: Azure AI Foundryで作成したプロジェクトのエンドポイントに接続し、デプロイされたGPTモデルを使用します。

2. **エージェントの作成**: 指定されたモデル（`gpt-4o-mini`など）を使用してAIエージェントを動的に作成し、日本語での親しみやすい対話を設定します。

3. **スレッド管理**: 会話を管理するためのスレッドを作成し、メッセージの履歴を適切に管理します。

4. **非同期処理**: エージェントの応答処理を非同期で実行し、ポーリングによってラン（実行）の完了を待機します。

5. **ユーザーとの対話**: ユーザーからのメッセージを送信し、エージェントの応答を取得・表示する完全な対話フローを実装します。

6. **リソース管理**: 使用後のエージェントリソースを適切にクリーンアップし、メモリリークを防止します。

### 注意事項

この例では、基本的なエージェント対話機能を実装しています。高度なRAG機能（ファイル検索、ベクトルストア）については、将来のバージョンで拡張予定です。現在は、Azure AI Agent Serviceの基本的な使用方法を学習できます。

### メイン関数

### 実行前の準備

Azure AI Agent Serviceを使用するには、以下の準備が必要です：

#### 1. Azure CLIでのログイン
```bash
az login
```

#### 2. 環境変数の設定
`.env`ファイルまたは環境変数に以下を設定してください：

```bash
# Azure AI Foundryプロジェクトのエンドポイント
PROJECT_ENDPOINT="https://your-project.services.ai.azure.com/api/projects/your-project-name"

# デプロイされたモデル名
MODEL_DEPLOYMENT_NAME="gpt-4o-mini"
```

#### 3. Azure AI Foundryプロジェクトの作成
- [Azure AI Foundry](https://ai.azure.com)でプロジェクトを作成
- GPT-4o-miniなどのモデルをデプロイ
- プロジェクトの「概要」→「Libraries」→「Azure AI Foundry」からエンドポイントをコピー
- 「Models + Endpoints」からモデルデプロイメント名をコピー

**注意**: このノートブックはAzure AI Foundryプロジェクトが必要です。設定されていない場合は、代替として`05-semantic-kernel-azuresearch.ipynb`の学習をお勧めします。

In [25]:
# Azure AI Agent Serviceの完全なRAGテスト実行（応答取得付き）
import time

async def complete_rag_test():
    """エージェントの応答を取得・表示する完全なテスト"""
    # Microsoft Learn標準の環境変数名を使用
    endpoint = os.getenv("PROJECT_ENDPOINT")
    model_deployment = os.getenv("MODEL_DEPLOYMENT_NAME")
    
    if not endpoint or not model_deployment:
        print("❌ 環境変数が設定されていません。")
        print("以下の環境変数を設定してください:")
        print("- PROJECT_ENDPOINT: Azure AI Foundryプロジェクトのエンドポイント")
        print("- MODEL_DEPLOYMENT_NAME: デプロイされたモデル名（例: gpt-4o-mini）")
        return
    
    try:
        # Azure AI Agents クライアントの作成
        agents_client = AgentsClient(
            endpoint=endpoint,
            credential=DefaultAzureCredential()
        )
        
        print("🤖 Azure AI Agent Service - 完全なRAGテストを開始します...")
        
        # エージェントの作成
        agent = agents_client.create_agent(
            model=model_deployment,
            name="RAGChatAgent",
            instructions="""あなたは親切で知識豊富なAIアシスタントです。
            ユーザーの質問に対して、明確で親しみやすい日本語で回答してください。
            自己紹介では、あなたの能力と制限について説明してください。""",
        )
        
        print(f"✅ エージェント '{agent.name}' を作成しました")
        
        # スレッドの作成
        thread = agents_client.threads.create()
        print(f"✅ 会話スレッドを作成しました")
        
        # テスト用のメッセージ
        user_message = "こんにちは！あなたは何ができますか？詳しく教えてください。"
        print(f"\n👤 ユーザー: {user_message}")
        
        # メッセージの作成
        message = agents_client.messages.create(
            thread_id=thread.id,
            role="user",
            content=user_message
        )
        
        # ランの作成と実行
        run = agents_client.runs.create(
            thread_id=thread.id,
            agent_id=agent.id
        )
        
        print("🔄 エージェントが処理中です...")
        
        # ランの完了を待機
        max_wait_time = 30  # 最大30秒待機
        start_time = time.time()
        
        while time.time() - start_time < max_wait_time:
            # ランの状態を取得
            current_run = agents_client.runs.get(
                thread_id=thread.id,
                run_id=run.id
            )
            
            if current_run.status == "completed":
                print("✅ エージェントの処理が完了しました")
                break
            elif current_run.status == "failed":
                print(f"❌ エージェントの処理が失敗しました: {current_run.last_error}")
                return
            elif current_run.status in ["cancelled", "expired"]:
                print(f"❌ ランが{current_run.status}になりました")
                return
            
            # 1秒待機してから再チェック
            time.sleep(1)
        else:
            print("⏰ タイムアウト: エージェントの応答に時間がかかりすぎています")
            return
        
        # エージェントの応答を取得
        messages = agents_client.messages.list(thread_id=thread.id)
        
        # 最新のアシスタントメッセージを表示
        for msg in reversed(list(messages)):
            if msg.role == "assistant":
                print(f"\n🤖 エージェント: {msg.content[0].text.value}")
                break
        
        # クリーンアップ
        agents_client.delete_agent(agent.id)
        print(f"\n🧹 エージェントをクリーンアップしました")
        
    except Exception as e:
        print(f"❌ エラーが発生しました: {str(e)}")
        print("\n解決方法:")
        print("1. Azure CLIで `az login` を実行してください")
        print("2. Azure AI Foundryプロジェクトが正しく設定されているか確認してください")
        print("3. 環境変数が正しく設定されているか確認してください")
        print("4. Azure AI User RBACロールがプロジェクトスコープで割り当てられているか確認してください")
        print("5. 最新のazure-ai-agentsパッケージがインストールされているか確認してください")

print("Azure AI Agent Service - 完全なRAGテスト（応答取得付き）を開始します...")
await complete_rag_test()

Azure AI Agent Service - 完全なRAGテスト（応答取得付き）を開始します...
🤖 Azure AI Agent Service - 完全なRAGテストを開始します...
✅ エージェント 'RAGChatAgent' を作成しました
✅ 会話スレッドを作成しました

👤 ユーザー: こんにちは！あなたは何ができますか？詳しく教えてください。
🔄 エージェントが処理中です...
✅ エージェントの処理が完了しました

🤖 エージェント: こんにちは！私はAIアシスタントです。あなたの質問に答えたり、情報を提供したり、さまざまな話題についてお手伝いすることができます。具体的には、以下のようなことが得意です。

1. **情報提供**: 歴史、科学、文化、技術など、さまざまな分野についての情報を提供できます。
2. **質問に答える**: 学校の宿題や日常生活の疑問など、さまざまな質問にお答えします。
3. **アドバイス**: 旅行の計画や勉強法、ライフスタイルに関するアドバイスを提供できます。
4. **文章作成**: エッセイや文章のアイデアを出したり、チェックしたりすることができます。

ただし、私は AI であり、感情を持っているわけではなく、個人の体験や意見を持っていません。また、情報は2023年10月までのものですので、それ以降のトピックについてはお答えできない場合があります。

どんなことでも気軽に質問してくださいね！

🧹 エージェントをクリーンアップしました


## 🎯 学習のポイント

### **このノートブックの主要な学習内容**:
- **Azure AI Agent Service**への接続と基本操作
- エージェントとの**実際の対話体験**
- **ラン完了の待機**とポーリング処理
- **エージェント応答の取得・表示**
- **基本的なエージェント管理ワークフロー**の実装

### **重要な技術要素**:
1. **ラン状態監視**: `agents_client.runs.get()` でstatusをポーリング
2. **応答取得**: `agents_client.messages.list()` で会話履歴を取得
3. **エラー処理**: タイムアウト、失敗、キャンセル状態の適切な処理
4. **リソース管理**: エージェントの適切なクリーンアップ
5. **Azure AI Agents SDK**: エージェント専用の最適化されたSDKの使用

### **次のステップ**:
- 実際のファイルアップロードとFileSearchToolを使ったRAG実装
- 複数の会話を続けるセッション管理
- カスタムツールの作成と統合